# Making maps with Folium

In [1]:
!pwd

/Users/trevorkinsey/reliance/trevor/mapping


In [3]:
import pandas as pd
import requests
import json
import folium
from folium.features import DivIcon
import overpy
import geopandas as gpd
import contextily as cx

from tqdm import tqdm

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

api = overpy.Overpass()

def get_nwr_in_bbox(south,west,north,east,key=None, value=None):
    """Gets all Nodes, Ways, Relations inside bounding box with key:value
    PARAMETERS:
        south,
        west,
        north,
        east,
        key
        value
    """
    if (key,value) != (None,None):
        query = f"""
        [out:json];
        nwr["{key}"="{value}"] ({south},{west},{north},{east});
        (._;>;);
        out body;"""
    else:
        query = f"""
        [out:json];
        nwr ({south},{west},{north},{east});
        (._;>;);
        out body;"""
    result = api.query(query)
    return result

In [4]:
# key="tourism"
# value="camp_site"
# result = get_nwr_in_bbox(lake["south"],lake["west"],lake["north"],lake["east"], key, value)
# len(result.nodes)

In [5]:
# # get ALL nodes, ways, relations
# key=None
# value=None
# # result = get_nwr_in_bbox(lake["south"],lake["west"],lake["north"],lake["east"], key, value)
# nodes = result.nodes
# ways = result.ways
# relations = result.relations
# print(f"{len(nodes)} nodes")
# print(f"{len(ways)} ways")
# print(f"{len(relations)} relations")

In [6]:
# # get all nodes and their tags (for those with tags)
# node_tags = {}
# for node in result.get_nodes():
#     if node.tags!={}:
#         node_tags[node.id] = node.tags
#         # print(node.id, node.tags)
# node_tags

In [7]:
def get_unique_tags(items):
    tags = {}
    for item in items:
        if item.tags!={}:
            for key,value in item.tags.items():
                if key not in tags:
                    tags[key] = [value]
                elif value not in tags[key]:
                    tags[key].append(value)
    return tags

In [23]:
node_tags = get_unique_tags(result.get_nodes())
node_tags

{'tourism': ['camp_site'],
 'name': ['Ahdatay',
  'Ahdatay Creek Provincial Site',
  "Miner's Camp",
  'Pine Point',
  'Progress Point',
  'Tchentlo Hot Springs Campsite',
  "Larry's Point",
  'Tchentlo']}

In [9]:
way_tags = get_unique_tags(result.get_ways())
way_tags

NameError: name 'result' is not defined

In [22]:
relation_tags = get_unique_tags(result.get_relations())
relation_tags

{}

In [8]:
# get all tag keys and values
tags = {}
for item in result.get_nodes():
    if item.tags!={}:
        for key,value in item.tags.items():
            if key not in tags:
                tags[key] = [value]
            elif value not in tags[key]:
                tags[key].append(value)
print(json.dumps(tags, indent = 4))

In [9]:
# for way in result.get_ways()[0:200]:
#     if way.tags!={}:
#         print(way.id, way.tags)

In [10]:
# for relation in result.get_relations():
#     if relation.tags!={}:
#         print(relation.id, relation.tags)

In [11]:
# # build a dict of nodes (lon,lat) for each way
# waydict={}
# for way in result.get_ways():
#     nodes = way.get_nodes(resolve_missing=True)
#     nodelist=[]
#     for node in nodes:
#         nodelist.append([float(node.lon), float(node.lat)])
#     waydict[way.id]=nodelist

# keys = list(waydict.keys())
# waydict[keys[0]]

In [13]:
def build_tiles():
    # folium tile layers
    tiles = {
        'Google Maps': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
            attr = 'Google', name = 'Google Maps', 
            overlay = False, control = True, show = True
        ),
        'Google Satellite': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            attr = 'Google', name = 'Google Satellite', 
            overlay = False, control = True, show = True
        ),
        'Google Terrain': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
            attr = 'Google', name = 'Google Terrain', 
            overlay = False, control = True, show = True
        ),
        'Google Satellite Hybrid': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
            attr = 'Google', name = 'Google Satellite',
            overlay = False, control = True, show = True
        ),
        'Esri Satellite': folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri', name = 'Esri Satellite',
            overlay = False, control = True, show = True
        ),
        'cartodbpositron': folium.TileLayer(
            tiles = 'cartodbpositron',
            overlay = False, control = True, show = True
        )
    }


    with open("stamen_api.json") as f:
        stamen = json.load(f)

    stamen_tiles = {name:folium.TileLayer(
        tiles = stamen["tiles"][name],
        attr = stamen["attribution"], 
        name = name,
        overlay = False, 
        control = True,
        show = True
    ) for name in stamen["tiles"]}

    tiles = tiles|stamen_tiles
    return tiles

tiles = build_tiles()
tile_names = list(tiles.keys())
tile_names


# Load nation lakes    
with open("tiles.pkl", "w") as f:
    dump(tiles, f)
tiles

TypeError: write() argument must be str, not bytes

# Get lake bounding boxes

In [16]:
lakes = [
    {
        "name": "Tsayta Lake",
        "north": 55.48292, 
        "west": -125.6440,
        "south": 55.41749450097668, 
        "east": -125.31534497067527,
    },
    {
        "name": "Indata Lake",
        "north": 55.3925, 
        "west": -125.32,
        "south": 55.292, 
        "east": -125.2175,
    },
    {
        "name": "Tchentlo Lake",
        "north": 55.27724688304605, 
        "west": -125.2879216313345,
        "south": 55.164542941519294, 
        "east": -124.75996931606227,
    },
    {
        "name": "Chuchi Lake",
        "north": 55.22210608559387, 
        "west": -124.76407610865795,
        "south": 55.14837905899054, 
        "east": -124.31095886659628,
    }
]
# get coordinates of lake centre 
for lake in lakes:
    lake["lat"] = (lake["north"] + lake["south"])/2
    lake["lon"] = (lake["east"] + lake["west"])/2

In [17]:
df = pd.DataFrame(lakes)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.crs = "WGS84" # web mercator 'EPSG:4326'
gdf.head()

,name,north,west,south,east,lat,lon,geometry
0,Tsayta Lake,55.482920,-125.644000,55.417495,-125.315345,55.450207,-125.479672,POINT (-125.47967 55.45021)
1,Indata Lake,55.392500,-125.320000,55.292000,-125.217500,55.342250,-125.268750,POINT (-125.26875 55.34225)
2,Tchentlo Lake,55.277247,-125.287922,55.164543,-124.759969,55.220895,-125.023945,POINT (-125.02395 55.22089)
3,Chuchi Lake,55.222106,-124.764076,55.148379,-124.310959,55.185243,-124.537517,POINT (-124.53752 55.18524)


In [18]:
print(f"tiles: {list(tiles.keys())}")

tiles: ['Google Maps', 'Google Satellite', 'Google Terrain', 'Google Satellite Hybrid', 'Esri Satellite', 'cartodbpositron', 'Stamen Toner', 'Stamen Toner Lite', 'Stamen Terrain', 'Stamen Watercolor']


In [19]:
lake = lakes[2]
font_color = "black"
tiles = build_tiles()

# set base tile layer
base_tile_name = 'Stamen Toner Lite'

#Create the map
map_centre = [lake["lat"], lake["lon"]]
zoom_level = cx.tile._calculate_zoom(lake["west"], lake["south"], lake["east"], lake["north"])
print(f"zoom_level={zoom_level}")
m = folium.Map(
    location = map_centre, 
    zoom_start = zoom_level-2, 
    min_zoom = 8, 
    max_zoom = 22, 
    tiles=None,
    control_scale=True,
    zoom_control=False,
)

# add tiles
for tile_name in tiles:
    tile_layer = tiles[tile_name]
    if tile_name == base_tile_name:
        tile_layer.overlay = False  # sets as base layer
        tile_layer.show = True      # shows on load
    else:
        tile_layer.overlay = True
        tile_layer.show = False
    if tile_name == "Stamen Terrain":
        tile_layer.opacity = 0.1
    tile_layer.add_to(m)
    
# add layer control element
folium.LayerControl().add_to(m)

# add centre marker
folium.CircleMarker(location=map_centre, radius=10, weight=2, color = "red").add_to(m)

# get campsites
result = get_nwr_in_bbox(
    lake["south"],lake["west"],lake["north"],lake["east"], 
    key="tourism", 
    value="camp_site")
nodes = result.nodes
print(f"{len(nodes)} nodes")

# add points at each node
for node in nodes:
    # location markers
    folium.CircleMarker(location=[node.lat, node.lon], radius=10, weight=2, color = "gray").add_to(m)
    # folium.Marker([node.lat, node.lon], icon = folium.Icon(color = "blue", icon='campground', prefix='fa')).add_to(m)
    
# Map title
icon=DivIcon(icon_size=(400,30), icon_anchor=(100,200), html=f'<div style="font-size: 40pt; color:{font_color}">{lake["name"]}</div>',)
folium.map.Marker([lake["lat"], lake["lon"]], icon=icon).add_to(m)
# m.fit_bounds([lake["west"], lake["south"]], [lake["east"], lake["north"]])
m

zoom_level=13
18 nodes


In [325]:
# Save to png

import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(f'{lake["name"]}.png')

In [20]:
!pwd

/Users/trevorkinsey/reliance/trevor/mapping


In [21]:
result = get_nwr_in_bbox(lake["south"],lake["west"],lake["north"],lake["east"], key="tourism", value="camp_site")


In [7]:
nodes

[<overpy.Node id=7579029096 lat=55.1844698 lon=-125.1106256>,
 <overpy.Node id=7579029097 lat=55.2018961 lon=-124.8032820>,
 <overpy.Node id=7579029098 lat=55.2224236 lon=-124.8436385>,
 <overpy.Node id=7579029099 lat=55.2105895 lon=-124.8862749>,
 <overpy.Node id=7579029100 lat=55.2265021 lon=-124.8984575>,
 <overpy.Node id=7579029101 lat=55.2206274 lon=-124.9142933>,
 <overpy.Node id=7579029102 lat=55.2135675 lon=-124.9466729>,
 <overpy.Node id=7579029103 lat=55.1913451 lon=-124.9725616>,
 <overpy.Node id=7579029105 lat=55.1962872 lon=-125.0631934>,
 <overpy.Node id=7579029106 lat=55.1838174 lon=-125.1274645>,
 <overpy.Node id=7579029107 lat=55.1961402 lon=-125.1348352>,
 <overpy.Node id=7579029108 lat=55.1816627 lon=-125.1413396>,
 <overpy.Node id=7579029109 lat=55.2023247 lon=-125.1553703>,
 <overpy.Node id=7579029110 lat=55.2004204 lon=-125.1911402>,
 <overpy.Node id=7579029111 lat=55.2193307 lon=-125.2340998>,
 <overpy.Node id=7579029114 lat=55.2294238 lon=-125.2479279>,
 <overpy

In [8]:
result.ways



[]

In [109]:
result.relations


[<overpy.Relation id=8515761>]

In [107]:
type(result)

overpy.Result

In [107]:
# Save to png

# import io
# from PIL import Image

# img_data = m._to_png(5)
# img = Image.open(io.BytesIO(img_data))
# img.save('folium-export.png')